# SETUP


In [ ]:
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, mean_squared_error

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# LOAD DATASETS

In [ ]:
dtypes = {
    'elapsed_time'  : np.int32,
    'event_name'    : 'category',
    'name'          : 'category',
    'level'         : np.uint8,
    'room_coor_x'   : np.float32,
    'room_coor_y'   : np.float32,
    'screen_coor_x' : np.float32,
    'screen_coor_y' : np.float32,
    'hover_duration': np.float32,
    'text'          : 'category',
    'fqid'          : 'category',
    'room_fqid'     : 'category',
    'text_fqid'     : 'category',
    'fullscreen'    : 'category',
    'hq'            : 'category',
    'music'         : 'category',
    'level_group'   : 'category'
}

TRAIN_DATASET_PATH = '/kaggle/input/student-performance-and-game-play/train.csv'
train_df = pd.read_csv(TRAIN_DATASET_PATH, dtype = dtypes)
print(f'train_df.shape = {train_df.shape}')
train_df.head()

In [ ]:
TRAIN_LABELS_DATASET_PATH = '/kaggle/input/student-performance-and-game-play/train_labels.csv'
train_labels_df = pd.read_csv(TRAIN_LABELS_DATASET_PATH)
print(f'train_labels_df.shape = {train_labels_df.shape}')
train_labels_df.head()

## PREPARE DATASETS

### Train dataset

In [ ]:
FEATURE_CATS = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']
FEATURE_NUMS = ['elapsed_time','level','page','room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration']

def feature_engineer(dataset, verbose = True):
    new_features = []

    temp = dataset.groupby(['session_id','level_group'])

    for feature in FEATURE_CATS:
        new_feature = temp[feature].agg('nunique')
        new_feature.name = feature + '_nunique'
        new_features.append(new_feature)

    for feature in FEATURE_NUMS:
        new_feature = temp[feature].agg('mean')
        new_features.append(new_feature)

    for feature in FEATURE_NUMS:
        new_feature = temp[feature].agg('std')
        new_feature.name = feature + '_std'
        new_features.append(new_feature)
        
    total_time_spent_feature = temp['elapsed_time'].agg('max') - temp['elapsed_time'].agg('min')
    total_time_spent_feature.name = 'total_time_spent'
    new_features.append(total_time_spent_feature)
    
    total_action_count_feature = temp['index'].agg('max') + 1
    total_action_count_feature.name = 'total_action_count'
    new_features.append(total_action_count_feature)
    
    average_time_spent_feature = total_time_spent_feature / total_action_count_feature
    average_time_spent_feature.name = 'average_time_spent'
    new_features.append(average_time_spent_feature)
        
    result = pd.concat(new_features, axis = 1)
    result = result.fillna(-1)
    result = result.reset_index()
    result = result.set_index('session_id')
    
    return result

In [ ]:
modified_train_df = feature_engineer(train_df)

del train_df
gc.collect()

modified_train_df.head()

### Train labels dataset

In [ ]:
train_labels_df['session']  = train_labels_df['session_id'].apply(lambda id: int(id.split('_q')[0]))
train_labels_df['question'] = train_labels_df['session_id'].apply(lambda id: int(id.split('_q')[1]))
train_labels_df.head()

## TRAIN MODEL

In [ ]:
session_ids = modified_train_df.index.unique()
split_index = int(len(session_ids) * 0.8) # train : validate = 8 : 2

train_session_ids = session_ids[:split_index]
valid_session_ids = session_ids[split_index:]

train_X_all_level_groups = modified_train_df.loc[train_session_ids]
valid_X_all_level_groups = modified_train_df.loc[valid_session_ids]

print(f'Train dataset size: {train_X_all_level_groups.shape[0]}')
print(f'Valid dataset size: {valid_X_all_level_groups.shape[0]}')

pred_df_by_max_depth   = {}
models_by_max_depth    = {}

max_depths = range(5, 16)
for max_depth in max_depths:
    pred_df = pd.DataFrame(data = np.zeros((len(valid_session_ids), 18)), index = valid_session_ids)
    
    models = {}
    
    for question in range(1, 19):
        if question <= 3: 
            level_group = '0-4'
        elif question <= 13:
            level_group = '5-12'
        elif question <= 18: 
            level_group = '13-22'
        
        train_X = train_X_all_level_groups[train_X_all_level_groups['level_group'] == level_group].drop('level_group', axis = 1)
        valid_X = valid_X_all_level_groups[valid_X_all_level_groups['level_group'] == level_group].drop('level_group', axis = 1)
        
        train_y = train_labels_df.loc[train_labels_df['question'] == question].set_index('session').loc[train_session_ids, 'correct']
        valid_y = train_labels_df.loc[train_labels_df['question'] == question].set_index('session').loc[valid_session_ids, 'correct']
    
        clf = RandomForestClassifier(max_depth = max_depth, n_estimators = 20, random_state = 42)
        clf.fit(train_X, train_y)
        
        class_index = list(clf.classes_).index(1)
        valid_y_pred = clf.predict_proba(valid_X)[:, class_index]
    
        pred_df.loc[valid_session_ids, question - 1] = valid_y_pred.flatten()
        models[f'{level_group}_{question}'] = clf
    
    pred_df_by_max_depth[max_depth] = pred_df
    models_by_max_depth[max_depth]  = models

In [ ]:
true_df = pd.DataFrame(data = np.zeros((len(valid_session_ids), 18)), index = valid_session_ids)
for question in range(1, 19):
    tmp = train_labels_df.loc[train_labels_df['question'] == question].set_index('session').loc[valid_session_ids]
    true_df[question - 1] = tmp['correct'].values
    
true_df.head()

In [ ]:
mse_by_max_depth = {}

best_mse_score = float('inf')
best_max_depth = -1

for max_depth in max_depths:
    score = mean_squared_error(true_df, pred_df)
    mse_by_max_depth[max_depth] = score
    
    if score < best_mse_score:
        best_mse_score = score
        best_max_depth = max_depth
        
pred_df = pred_df_by_max_depth[best_max_depth]
models = models_by_max_depth[best_max_depth]        

print(f'Best max depth :    {best_max_depth}')
print(f'Best MSE       :    {best_mse_score}')

In [ ]:
def find_best_threshold(pred, true, min = 0.4, max = 0.81, step = 0.01):
    scores = []
    best_score = 0
    best_threshold = 0
    
    true_label = true.to_numpy().flatten()
  
    thresholds = np.arange(min, max, step)
    for threshold in thresholds:
        predict = (pred.to_numpy().flatten() > threshold).astype('int')
        m = f1_score(true_label, predict, average = 'macro')
        scores.append(m)
        if m > best_score:
            best_score = m
            best_threshold = threshold

    plt.figure(figsize = (20, 5))
    plt.scatter([best_threshold], [best_score], color = 'g', s = 300)
    plt.plot(thresholds, scores, marker = 'o', color = 'g')
    plt.xlabel('Threshold')
    plt.ylabel('F1 Score')
    plt.title(f'Threshold vs F1 Score (best F1 Score = {best_score:.4f}, best threshold = {best_threshold:.4f})')
    plt.show()

    return best_threshold, best_score

best_threshold, best_score = find_best_threshold(pred_df, true_df)
print(f'Threshold :    {best_threshold}')
print(f'F1-Score  :    {best_score}')

In [ ]:
modified_pred_df = (pred_df > best_threshold).astype(int)
modified_pred_df.head()

In [ ]:
confusion = confusion_matrix(true_df.values.reshape((-1)), modified_pred_df.values.reshape((-1)))
plt.figure(figsize = (8, 6))
sns.heatmap(confusion, annot = True, fmt = 'd') 
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
for question in range(1, 19):
    m = f1_score(true_df[question - 1].values, (pred_df[question - 1].values > best_threshold).astype('int'), average = 'macro')
    print(f'Question {question:2}: F1 = ', m)
  
m = f1_score(true_df.values.reshape((-1)), (pred_df.values.reshape((-1)) > best_threshold).astype('int'), average = 'macro')
print('==> Overall F1 =', m)

## SUBMISSION

In [ ]:
import jo_wilder
env = jo_wilder.make_env()
iter_test = env.iter_test()

limits = {'0-4': (1, 4), '5-12': (4, 14), '13-22': (14, 19)}

for (test, sample_submission) in iter_test:
    test_df = feature_engineer(test, verbose = False)
    for level_group in test_df['level_group'].unique():
        test_ds = test_df[test_df['level_group'] == level_group].drop('level_group', axis = 1)
        if test_ds.shape[0] > 0:
            a, b = limits[level_group]
            for question in range(a, b):
                model = models[f'{level_group}_{question}']
                class_idx = list(model.classes_).index(1)
                mask = sample_submission['session_id'].str.endswith(f'q{question}')
                sample_submission.loc[mask,'correct'] = (model.predict_proba(test_ds)[:, class_idx] > best_threshold).astype('int').flatten()

    env.predict(sample_submission)

In [ ]:
! head submission.csv